# **Assignment 3**

## **ECON8502 - Structural Microeconometrics**

### Conor Bayliss

#### **Setup**

In [2]:
using Optim, Distributions, Statistics, CSV, DataFrames, Plots, DataFramesMeta, LinearAlgebra

##### **Source code for the model**

In [3]:
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model.jl")
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model\\choices.jl")
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model\\utility.jl")
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model\\states_transitions.jl")
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model\\solve.jl")

plain_logit (generic function with 1 method)

##### **Setting up exogenous state variables**

In [4]:
struct model_data
    T::Int64 #<- length of problem

    y0::Int64 #<- year to begin problem
    age0::Int64 # <- mother's age at start of problem
    SOI::Vector{Int64} #<- state SOI in each year
    num_kids::Vector{Int64} #<- number of kids in household that are between age 0 and 17
    TotKids::Int64 #<- indicares the total number of children that the mother will have over the available panel
    age_kid::Matrix{Int64} #< age_kid[f,t] is the age of child f at time t. Will be negative if child not born yet.
    cpi::Vector{Float64} #<- cpi

    R::Vector{Int64} #<- indicates if work requirement in time t
    Kω::Int64 #<- indicates length of time limit once introduced
    TL::Vector{Bool} #<- indicating that time limit is in place
end

In [5]:
md = test_model()
p = pars(5,5) #<- set Kτ = 5 and Kε = 5

(Kτ = 5, Kε = 5, β = 0.98, αC = 1.0, αθ = [0.1, 0.1, 0.1, 0.1, 0.1], αH = [0.0, 0.0, 0.0, 0.0, 0.0], αA = [0.0, 0.0, 0.0, 0.0, 0.0], αS = [0.0, 0.0, 0.0, 0.0, 0.0], αR = [0.0, 0.0], σ = [2.0, 2.0], σ_W = 2.0, σε = 2.0, βΓx = [0.0, 0.0], βΓτ = [0.0, 0.0], βw = [6.0 6.375 … 7.125 7.5; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], βτ = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], πε = [0.2 0.2 … 0.2 0.2; 0.2 0.2 … 0.2 0.2; … ; 0.2 0.2 … 0.2 0.2; 0.2 0.2 … 0.2 0.2], εgrid = LinRange{Float64}(-1.0, 1.0, 5), αl = 1, πW = 0.8, ση = 2.0)

##### **Nested logit probabilities**

In [6]:
B₁ = [[1,2],[3,4],[5,6]]
C₁ = [[1,],[2,],[3,],[4,],[5,],[6,]]

B₂ = [[1,2,3]]
C₂ = [[1,2],[3,4],[5,6]]

B = (B₁,B₂)
C = (C₁,C₂)

([[1], [2], [3], [4], [5], [6]], [[1, 2], [3, 4], [5, 6]])

##### **Indexing the state**

In [7]:
# Hypothetical state space dimensions:
Kε = 5
Kτ = 5
Kω = 6

k_idx = LinearIndices((Kτ,Kε,Kω))
k_inv = CartesianIndices(k_idx)

# To get the aggregate index k, call:
k = k_idx[2,3,2]
@show k
# Then if we have k we can work back with:
k_tuple = Tuple(k_inv[k])
@show k_tuple

k = 37
k_tuple = (2, 3, 2)


(2, 3, 2)

And then create the corresponding `NamedTuple`. This will come in handy later.

In [8]:
# Create the NamedTuple
state_idx = (Kε=Kε, Kτ=Kτ, Kω=Kω, k_idx=k_idx, k_inv=k_inv)

(Kε = 5, Kτ = 5, Kω = 6, k_idx = [1 6 … 16 21; 2 7 … 17 22; … ; 4 9 … 19 24; 5 10 … 20 25;;; 26 31 … 41 46; 27 32 … 42 47; … ; 29 34 … 44 49; 30 35 … 45 50;;; 51 56 … 66 71; 52 57 … 67 72; … ; 54 59 … 69 74; 55 60 … 70 75;;; 76 81 … 91 96; 77 82 … 92 97; … ; 79 84 … 94 99; 80 85 … 95 100;;; 101 106 … 116 121; 102 107 … 117 122; … ; 104 109 … 119 124; 105 110 … 120 125;;; 126 131 … 141 146; 127 132 … 142 147; … ; 129 134 … 144 149; 130 135 … 145 150], k_inv = CartesianIndices((5, 5, 6)))

##### **Transition probabilities**

In [9]:
fε(3,5,0.9)

((2, 3, 4), (0.04999999999999999, 0.9, 0.04999999999999999))

In [10]:
fε(1,5,0.9)

((1, 2), (0.95, 0.04999999999999999))

It is important to know how $j$ is indexed in the code. It is indexed via the `j_idx` function in `choices.jl`.
* $j = 1$ : $\quad$ $s = 0$, $A = 0$, $H = 0$
* $j = 2$ : $\quad$ $s = 0$, $A = 0$, $H = 1$
* $j = 3$ : $\quad$ $s = 1$, $A = 0$, $H = 0$
* $j = 4$ : $\quad$ $s = 1$, $A = 0$, $H = 1$
* $j = 5$ : $\quad$ $s = 1$, $A = 1$, $H = 0$
* $j = 6$ : $\quad$ $s = 0$, $A = 0$, $H = 1$

# **TO DO**

Also, we need to keep the structure of the model in mind. Recall that $B_1$ = $\left[\left[1,2\right],\left[3,4\right],\left[5,6\right]\right]$ indexes the possible choices in the second layer, given that you have chosen one of $C_2$ = $\left[\left[1,2\right],\left[3,4\right],\left[5,6\right]\right]$. 

# **TO DO**

#### **Question 1**

Write a function `calc_vj` which calculates the choice-specific value (i.e. the deterministic value of the choice) of a particular choice $j$ in a particular time period $t$ given the state and other exogenous variables. If you are confident you can code this however you like, but given the existing setup you might like to write the function in a way that takes the following inputs:

* `j`: the discrete choice
* `t`: the time period in the model
* `state`: a tuple that contains the state $(k_{\tau}, k_{\epsilon}, k_{\omega})$ as well as a linear indexing rule
* `V`: a vector that contains the continuation value for each state at time `t+1`
* `pars`: the parameters of the model
* `md`: an instance of the `model_data` object that holds all relevant state variables

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.


In [11]:
function calc_vj(j,t,state,V,pars,md)
    (;β,αC,αθ,αH,αA,αS,αR,σ,σ_W,σε,βΓx,βΓτ,βw,βτ,πε,εgrid,αl,πW,ση) = pars
    S,A,p,H = j_inv(j)
    kτ, kε, kω = state
    next_state, transition_prob = fε(kε, Kε, πW)
    return utility(S,A,H,pars,md,kτ,kε,kω,t)  + sum(β.*(V[collect(next_state)] .* transition_prob))
end

calc_vj (generic function with 1 method)

Now, let's test our `calc_vj` function to calculate a single element.

In [12]:
test_j = 2
test_t = 1
test_state = [1,1,1]

V = zeros(Float64, 6)

@time begin
    vj_out = calc_vj(test_j,test_t,test_state,V,p,md)
end


  0.231715 seconds (316.08 k allocations: 20.954 MiB, 99.84% compilation time)


9.611080473485494

Now calculate a vector of values.

In [73]:
vj_test = zeros(Float64,6)
@time begin
    for j in 1:6
        vj_test[j] = calc_vj(j,5,[1,1,1],V,p,md)
    end
end

@show vj_test

6-element Vector{Float64}:
 -Inf
   9.702051394510077
  11.061640646096055
  11.08632044958079
  12.304304815389033
  12.074036876375626

#### **Question 2**

Write a function called `iterate!` that iterates over all states at time period $t$ and fills in choice probabilities and continuation values for period $t$ in pre-allocated arrays. Again, you can do this however you like but here is a suggested set of inputs:

* `t`: the time period
* `logP`: a $J$ x $K$ x $T$ array of choice probabilities where the function will fill in `logP[:,:,t]`
* `V`: a $K$ x $T$ array of continuation values
* `state_idx`: a named tuple that contains the size of the overall state space, a linear indexing rule that maps $(k_{\tau}, k_{\epsilon}, k_{\omega})$ to an overall state $k$, and a Cartesian Indexing rule that inverts the mapping
* `vj`: a $J$-dimensional vector that, for each state, can be used as a container for the choice-specific values
* `pars`: model parameters
* `md`: `model_data` for the problem

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.

In [75]:
function iterate!(t,logP,V,state_idx::NamedTuple,vj,pars,md::model_data)
    (;σ) = pars ### not sure which pars are needed here
    prob = zeros(Float64,6)
    expvj = zeros(Float64,6)
    IV = zeros(Float64,3)
    P = zeros(Float64,6)
    for j in 1:6
        if vj[j] == -Inf
            expvj[j] = 0
        else    
            expvj[j] = exp(vj[j]/σ[1])
        end
    end
    #@show expvj
    for j in 1:6
        prob[j] = exp.(vj[j]) ./ sum(exp.(vj))
    end
    @show prob
    IV[1] = σ[1] * log.(sum(expvj[1:2]/σ[1]))
    IV[2] = σ[1] * log.(sum(expvj[3:4]/σ[1]))
    IV[3] = σ[1] * log.(sum(expvj[5:6]/σ[1]))
    sumIV = sum(exp.(IV/σ[2]))
    nest_probs = exp.(IV/σ[2]) ./ sumIV
    P[1] = nest_probs[1] *  prob[1]
    P[2] = nest_probs[1] *  prob[2]   
    P[3] = nest_probs[2] *  prob[3]
    P[4] = nest_probs[2] *  prob[4]
    P[5] = nest_probs[3] *  prob[5]
    P[6] = nest_probs[3] *  prob[6]
    V[t] = σ[2]*log(sum(expvj))
    logP = log.(P)
    @show logP
    return V, logP
end

iterate! (generic function with 1 method)

In [76]:
md = test_model()
V = zeros(Float64, md.T)
logP = zeros(Float64, 6)
vj = zeros(Float64,6)

function test_iterate(V, logP, state_idx::NamedTuple, vj, p, md::model_data)
    state = [1,1,1]
    @show state
        for t in md.T:-1:1
            for j in 1:6
                vj[j] = calc_vj(j,t,state_idx,V,p,md)
            end
        V, logP = iterate!(t,logP,V,state_idx,vj,p,md)
    end
    return V, logP
end

test_iterate (generic function with 1 method)

In [77]:
v, log_p = test_iterate(V_big, logP, state_idx, vj, p, md)

([866.6826561222578 866.7242893366171 … 0.0 0.0; 866.7070385147728 866.7280090742709 … 0.0 0.0; … ; 866.7177689936016 866.7485830962326 … 0.0 0.0; 866.7207303061796 866.7620406349148 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 

#### **Question 3**

Write a function called `solve!` that performs backward induction to calculate continuation values and choice probabilities (storing them in pre-allocated arrays) in every period of the data across the whole state space. As before, some suggested inputs:

* `logP`: a $J$x$K$x$T$ array for the choice probabilities 
* `V`: a $K$ x $T$ array for continuation values
* `vj`: a container or buffer for choice-specific values in each iteration
* `pars`: model parameters
* `md`: an instance of `model_data`

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.

In [17]:
V_big = zeros(Float64, Kτ, Kε, Kω, md.T) #<- this is the value function
@show size(V_big) # <- should be Kτ (types) x Kε (wage grid) x Kω (time use) x T (time periods)

size(V_big) = (5, 5, 6, 12)


(5, 5, 6, 12)

In [18]:
function solve!(logP,V,vj,pars,md::model_data)
    for t in md.T:-1:1
        for k in eachindex(k_idx)
            state = Tuple(k_inv[k])
            for j in 1:6
                vj[j] = calc_vj(j,t,state,V,pars,md)
            end
            iterate!(t,logP,V,state,vj,pars,md)
        end
    end
end

solve!(logP,V_big,vj,p,md)

MethodError: MethodError: no method matching iterate!(::Int64, ::Vector{Float64}, ::Array{Float64, 4}, ::Tuple{Int64, Int64, Int64}, ::Vector{Float64}, ::@NamedTuple{Kτ::Int64, Kε::Int64, β::Float64, αC::Float64, αθ::Vector{Float64}, αH::Vector{Float64}, αA::Vector{Float64}, αS::Vector{Float64}, αR::Vector{Float64}, σ::Vector{Float64}, σ_W::Float64, σε::Float64, βΓx::Vector{Float64}, βΓτ::Vector{Float64}, βw::Matrix{Float64}, βτ::Matrix{Float64}, πε::Matrix{Float64}, εgrid::LinRange{Float64, Int64}, αl::Int64, πW::Float64, ση::Float64}, ::model_data)

Closest candidates are:
  iterate!(::Any, ::Any, ::Any, !Matched::NamedTuple, ::Any, ::Any, ::model_data)
   @ Main c:\Users\bayle\Documents\Github\micro_labour\hw3\hw3.ipynb:1
